In [1]:
# Импортируем либы
from pyspark.sql import SparkSession

import pyspark.sql.functions as F

In [2]:
# Создаем сессию

spark = (
    SparkSession.builder
        .master("local")
        .appName("Basics-operations")
        .getOrCreate()
)

In [7]:
# Считываем данные

df = spark.read.parquet("data/nyc_taxi/")

In [10]:
# Информация о schema

df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- filename: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



### SELECT — выбор колонок

In [ ]:
# SELECT col1, col2 FROM df;

df.select()